### Creating Passage Chunk

In [28]:
import os
import pandas as pd
data_dir = "./Data/"
metadata_path = os.path.join(data_dir,"metadata.csv")
metadata_df = pd.read_csv(metadata_path,
                               dtype={'Microsoft Academic Paper ID': str, 'pubmed_id': str})
metadata_df = metadata_df.dropna(subset=['abstract', 'title']).reset_index(drop=True)
metadata_df = metadata_df.drop_duplicates(['abstract', 'title']).reset_index(drop = True)

#Subsetting Columns
final_metadata = metadata_df[['abstract', 'title']]
final_metadata["id"] = [str(i) for i in range(final_metadata.shape[0])]

C:\Users\bansa\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (1,4,5,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\bansa\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [7]:
from nltk.tokenize import sent_tokenize
import numpy as np
from transformers import AutoTokenizer
TOKENIZER = AutoTokenizer.from_pretrained('./pre_trained_model/training_nli_covidbert-mednli/0_Transformer')
MAX_LEN = 300
STRIDE = 1

In [8]:
def get_para_segments(text, stride, max_len, id_, title, tokenizer):
    """
    Get Running length window of certain length with a paritcular stride
    """
#     tokenizer = AutoTokenizer.from_pretrained('./pre_trained_model/training_nli_covidbert-mednli/0_Transformer')
    text_map = {i:sent for i, sent in enumerate(sent_tokenize(text))}
    text_lenmap = {i:len(input_id) for i,input_id in enumerate(tokenizer(list(text_map.values()))['input_ids'])}
    
    para = []
    i = 0
    if len(text_map) > 1:
        while i < len(text_map):
            for j in text_map.keys():
                if j > i:
                    new_para_sub_len = np.sum(list(text_lenmap.values())[i:j])
                    if j == (len(text_map) -1):
                        para.append("".join(list(text_map.values())[i:(j+1)]))
                        i = 999999 # some big value
                    if new_para_sub_len <= max_len:
                        continue
                    else:                                                 
                        para.append( "".join(list(text_map.values())[i:j]))
                        i = i+stride
    else:
        para.append(text_map[0])
        
    # at least 5 words should be there in the paragraph
    # para = [paragraph for paragraph in para if len(paragraph.split()) > 5]
    return [[id_, str(id_) + "_" + str(i), title, paragraph] for i,paragraph in enumerate(para)]
    
    

In [9]:
# from tqdm import tqdm
# import pickle

# for i,df in enumerate(np.array_split(final_metadata, 10)):
#     print(i)
#     passage_list = [get_para_segments(row["abstract"],STRIDE, MAX_LEN,row["id"],
#                                       row["title"], TOKENIZER) for i,row in tqdm(df.iterrows())]
#     with open('./Data/passage/passage_'+str(i)+'.pkl', 'wb') as f:
#         pickle.dump(passage_list, f)
#         del passage_list

### Bert QE

In [1]:
import glob
import pickle
import pandas as pd
all_metadata = []
for i,files in enumerate(glob.glob("./Data/passage/passage_*.pkl")):
    with open(files, 'rb') as f:
        data_list = pickle.load(f)
        all_metadata.extend([data_pair for data in data_list \
                             for data_pair in data])

all_metadata_df = pd.DataFrame(all_metadata, columns = ["id","passage_id","title","passage"])

In [2]:
# import json
# json_lines = [{'id': row['passage_id'], 'contents': row['passage']} for i,row in all_metadata_df.iterrows()]

# with open("./Data/passage_data.jsonl", 'w') as f:
#     for lines in json_lines:
#         f.write(json.dumps(lines) + "\n")

In [3]:
from pyserini.search import SimpleSearcher
bm25 = SimpleSearcher('./Data/indexes')

# search_hits = bm25.search('what is coronavirus', k= 200)
# bm25_passage = [hit.docid for hit in search_hits]

In [4]:
from rank_bm25 import BM25Okapi
import numpy as np
import nltk
from itertools import chain

In [5]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()
# Create a blank Tokenizer with just the English vocab
tokenizer = Tokenizer(nlp.vocab)

In [6]:
class BM25RankedResults:
    """
    BM25 Results from the abstract.
    
    Usage:
    
    bm25 = BM25RankedResults(metadata_df) # metadata_df is a pandas dataframe with 'title' and 'abstract' columns 
    topbm25 = bm25.search("What is coronavirus", num=10) # Return `num` top-results
    """

    def __init__(self, corpus: pd.DataFrame):
        self.corpus = corpus
        self.columns = corpus.columns
        token_list = pd.Series([[str(token) for token in doc if str(token)] \
         for doc in tokenizer.pipe(corpus.passage, 
                                   batch_size=5000)])
        self.index = token_list.to_frame()
        self.index.columns = ['terms']
        self.index.index = self.corpus.index
        self.bm25 = BM25Okapi(self.index.terms.tolist())
        
        self.bm25 = BM25Okapi(token_list)
    
    def search(self, query, num = 200):
        """
        Return top `num` results that better match the query
        """
        search_terms = query.split() 
        doc_scores = self.bm25.get_scores(search_terms) # get scores
        
        ind = np.argsort(doc_scores)[::-1][:num] # sort results
        
        results = self.corpus.iloc[ind][self.columns] # Initialize results_df
        results['score'] = doc_scores[ind] # Insert 'score' column
        results = results[results.score > 0]
        return results.passage_id.tolist()

In [7]:
# passage_data = all_metadata_df.sample(5000)
# bm25_okapi = BM25RankedResults(passage_data) 

In [8]:
from sentence_transformers import SentenceTransformer,util
covid_bert =  SentenceTransformer("deepset/covid_bert_base", device = 'cuda')


Exception when trying to download https://sbert.net/models/deepset/covid_bert_base.zip. Response 404
SentenceTransformer-Model https://sbert.net/models/deepset/covid_bert_base.zip not found. Try to create it from scratch
Try to create Transformer Model deepset/covid_bert_base with mean pooling


In [9]:
def get_top_k_vals(list1, list2, k = 100, model = covid_bert, return_cosine_mat = False):
    # Compute embedding for both lists
    embeddings1 = model.encode(list1, convert_to_tensor = True)
    embeddings2 = model.encode(list2, convert_to_tensor = True)
    
    # Compute cosine-similarity
    cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
    
    if return_cosine_mat:
        return cosine_scores.numpy()
    
    # Select top kd documents/passage
    _topkd = np.argsort(cosine_scores.numpy()[0])[::-1][:k]
    
    return _topkd, cosine_scores.numpy()[0][_topkd]
    

In [10]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [11]:
from collections import OrderedDict
def bert_qe(query, bm25_model, passage_id_map, bert_model = covid_bert, 
            alpha = 0.4, document_size = 500, chunk_size = 8):
    """
    Re-ranks BM-25 document based on relevancy of query to chunks of a passage.
    """
    
    print("\tPhase 1")
    # Phase 1
    topbm25 = bm25_model.search(query, document_size)
    
    #doc index to passage map
    passage_index_map = OrderedDict({idx:passage_id_map[passages] if isinstance(passages,str) \
                               else passage_id_map[passages.docid] for idx,passages in enumerate(topbm25)})
    passageid_index_map = OrderedDict({idx:passages if isinstance(passages,str) \
                               else passages.docid for idx,passages in enumerate(topbm25)})
    
    _topdocidx, _topdocscores = get_top_k_vals([query], 
                                         list(passage_index_map.values()),
                                         k = document_size, model = bert_model)
    # Store Top Contextually matching docs
    passage_scores = {idx:score for idx,score in zip(_topdocidx, _topdocscores)}
    
    
    print("\tPhase 2")
    # Phase 2
    # Create chunks of length "n" and stride them with a length of "n/2"
    _chunks = [[" ".join(phrase) for i, phrase in enumerate(nltk.ngrams(passage_index_map[idx].split(), 
                                                                        chunk_size)) if i%(chunk_size/2)==0] for idx in _topdocidx]
    
    # Flatten the list
    all_chunks = list(chain.from_iterable(_chunks))
    
    # Get top chunks based on relevancy score with the query
    _topchunkidx, _topchunkscores = get_top_k_vals([query], 
                                         all_chunks, 
                                         k = int(len(all_chunks)/2), model = bert_model)
    
    top_chunks = np.array(all_chunks)[_topchunkidx]
    
    # Apply softmax over query and chunk relevancy score, 
    # This acts as weights to chunk and document relevancy
    _topchunksoftmax = softmax(_topchunkscores)
    
    
    # Phase 3
    print("\tPhase 3")
    scores = get_top_k_vals(list(passage_index_map.values()), 
                            list(top_chunks), 
                            k = len(top_chunks), 
                            model = bert_model, 
                            return_cosine_mat = True)
    
    # Multiply the weights of chunk with query to relevancy of chunk with the document
    # and sum over all the top chunks (kc in the paper)
    docchunk_score = np.sum(np.multiply(_topchunksoftmax, np.array(scores)), axis = 1)
         
    # weighing importance of query relevance and query chunk-doc relevance
    
    final_score = alpha*_topdocscores + (1-alpha)*docchunk_score
    
    passage_score = dict(zip([passageid_index_map[idx] for idx in _topdocidx],final_score))

    return passage_score

## Semantic Search

In [12]:
# Instantiate the sentence-level covid-BERT NLI model
from sentence_transformers import SentenceTransformer,util
covid_nli = SentenceTransformer('./pre_trained_model/training_nli_covidbert-mednli', device = 'cuda')


In [13]:
# # Convert abstracts to vectors
# embeddings = covid_nli.encode(all_metadata_df.passage.to_list()[:100], show_progress_bar=True)

In [14]:
import faiss
import numpy as np

In [15]:
# # Building FAISS Index
# embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

# # Instantiate the index
# embedding_index = faiss.IndexFlatIP(embeddings.shape[1])

# # Pass the passage index to IndexIDMap
# embedding_index = faiss.IndexIDMap(embedding_index)

# # Numerical map
# passage_num_map = {i:x for i,x in enumerate(all_metadata_df.passage_id.values)}

# # Add vectors and their IDs
# embedding_index.add_with_ids(embeddings, np.array(list(passage_num_map.keys()), np.int64))

In [16]:
embedding_index = faiss.read_index("./Data/faiss_cord-19-passage.index")

## Re-Ranking on MedMarco

In [17]:
from sentence_transformers.cross_encoder import CrossEncoder
covid_marco = CrossEncoder("./pre_trained_model/training_medmarco_covidbert", device = 'cuda')

# # Check if GPU is available and use it
# if torch.cuda.is_available():
#     covid_marco = covid_marco.to(torch.device("cuda"))
# print(covid_marco.device)

In [18]:
def get_ranked_passages(query, bm25_model, bert_model, passage_id_map, faiss_index, bert_qe_alpha = 0.4):
    
    print("Step 1 : BERT-QE Expansion")
    #BERT-QE
    bertqe_dict = bert_qe(query, bm25_model = bm25_model, passage_id_map = passage_id_map,
        bert_model = bert_model, alpha = 0.4, document_size = 500, chunk_size = 8)
    
    print("Step 2 : Semantic Passage Retrieval")
    # Semantic Search
    _,indices = faiss_index.search(np.expand_dims(covid_nli.encode(query), axis = 0), k=500)
    semantic_passage_ids = [passage_num_map[idx] for idx in indices[0]]
    
    # passages to be re-ranked
    total_passage_ids = list(bertqe_dict.keys())+ semantic_passage_ids
    
    return list(set(total_passage_ids))
    

### Comprehension

In [19]:
# model used for comprehension
from transformers import pipeline
comprehension_model = pipeline("question-answering", model='graviraja/covidbert_squad',
                               tokenizer='graviraja/covidbert_squad', device=-1,use_fast = True)

In [20]:
covid_kaggle_questions = [
          {
              "task": "What is known about transmission, incubation, and environmental stability?",
              "questions": [
                  "Is the virus transmitted by aerosol, droplets, food, close contact, fecal matter, or water?",
                  "How long is the incubation period for the virus?",
                  "Can the virus be transmitted asymptomatically or during the incubation period?",
                  "How does weather, heat, and humidity affect the transmission of 2019-nCoV?",
                  "How long can the 2019-nCoV virus remain viable on common surfaces?"
              ]
          },
          {
              "task": "What do we know about COVID-19 risk factors?",
              "questions": [
                  "What risk factors contribute to the severity of 2019-nCoV?",
                  "How does hypertension affect patients?",
                  "How does heart disease affect patients?",
                  "How does copd affect patients?",
                  "How does smoking affect patients?",
                  "How does pregnancy affect patients?",
                  "What is the fatality rate of 2019-nCoV?",
                  "What public health policies prevent or control the spread of 2019-nCoV?"
              ]
          },
          {
              "task": "What do we know about virus genetics, origin, and evolution?",
              "questions": [
                  "Can animals transmit 2019-nCoV?",
                  "What animal did 2019-nCoV come from?",
                  "What real-time genomic tracking tools exist?",
                  "What geographic variations are there in the genome of 2019-nCoV?",
                  "What efforts are being done in asia to prevent further outbreaks?"
              ]
          },
          {
              "task": "What do we know about vaccines and therapeutics?",
              "questions": [
                  "What drugs or therapies are being investigated?",
                  "Are anti-inflammatory drugs recommended?"
              ]
          },
          {
              "task": "What do we know about non-pharmaceutical interventions?",
              "questions": [
                  "Which non-pharmaceutical interventions limit transmission?",
                  "What are the most important barriers to compliance?"
              ]
          },
          {
              "task": "What has been published about medical care?",
              "questions": [
                  "How does extracorporeal membrane oxygenation affect 2019-nCoV patients?",
                  "What telemedicine and cybercare methods are most effective?",
                  "How is artificial intelligence being used in real time health delivery?",
                  "What adjunctive or supportive methods can help patients?"
              ]
          },
          {
              "task": "What do we know about diagnostics and surveillance?",
              "questions": [
                  "What diagnostic tests (tools) exist or are being developed to detect 2019-nCoV?"
              ]
          },
          {
              "task": "Other interesting questions",
              "questions": [
                  "What is the immune system response to 2019-nCoV?",
                  "Can personal protective equipment prevent the transmission of 2019-nCoV?",
                  "Can 2019-nCoV infect patients a second time?"
              ]
          }
   ]


In [23]:
# covid_kaggle_questions = [{'task':'others','questions':["How does covid 19 affect smokers",
#                                                        "what are common symptoms observed in covid 19 patient",
#                                                        "how does co-existing viral infections affect transmission",
#                                                        "What do we know about covid 19 in pregant womens"]},
#                          {
#               "task": "Other interesting questions",
#               "questions": [
#                   "What is the immune system response to 2019-nCoV",
#                   "Can personal protective equipment prevent the transmission of 2019-nCoV",
#                   "Can 2019-nCoV infect patients a second time"
#               ]
#           }]

In [24]:
# Map of Passage id to Passage Text
passage_id_map = pd.Series(all_metadata_df.passage.values,index=all_metadata_df.passage_id).to_dict()

# Numerical map for semantic passage retreival
passage_num_map = pd.Series(all_metadata_df.passage_id.values,index=pd.Series(range(len(all_metadata_df)))).to_dict()

# Map of Passage id to Paper Title
passage_id_title_map = pd.Series(all_metadata_df.title.values,index=all_metadata_df.passage_id).to_dict()


all_comprehension_df_list = []
#Search in a loop for the individual queries
for task_query_dict in covid_kaggle_questions:
    for i,query in enumerate(task_query_dict["questions"]):
        print(f"Ranking Passages for {i+1} of {len(task_query_dict['questions'])} query/queries")
        passage_ids = get_ranked_passages(query, bm25_model = bm25, passage_id_map = passage_id_map,
                                          bert_model = covid_bert,
                                          faiss_index = embedding_index, bert_qe_alpha = 0.4)


        #Concatenate the query and all passages and predict the scores for the pairs [query, passage]
        model_inputs = [[query, passage_id_map[passage_id]] for passage_id in passage_ids]

        print("Step 3 : Passage Re-ranking using Fine-Tuned Covid BERT ")
        scores = covid_marco.predict(model_inputs)

        #Sort the scores in decreasing order
        results = [{'input': inp, 'score': score} for inp, score in zip(passage_ids, scores)]
        results = sorted(results, key=lambda x: x['score'], reverse=True)

        # Filtering passages above a certain threshold
        final_results = {res_dict['input']:res_dict['score'] for res_dict in results if res_dict['score'] > 0.3}
        
        print("Step 4 : Comprehension ")
        # Comprehension
        comp_output = [[comprehension_model(question="What is coronavirus", 
                                     context = passage_id_map[pass_id]),  pass_id, pass_score] \
                  for pass_id, pass_score in final_results.items() if len(passage_id_map[pass_id].split()) > 5] 

        # Adding pass id and score to the comprehension
        [comp_output[i][0].update({'pass_id': comp_output[i][1], 
                             'pass_rank_score': comp_output[i][2]}) for i in range(len(comp_output))]
        
        # Converting list of dictionaries of ranked results to dataframe.
        comprehension_df = pd.DataFrame([comp_[0] for comp_ in comp_output])
        
        # adding query and the task
        comprehension_df["query"] = query
        comprehension_df["task"] = task_query_dict["task"]
        
        # Finally, using passage_id to replace with actual Paper Title and Context
        comprehension_df["title"] = [passage_id_title_map[pass_id] for pass_id in comprehension_df.pass_id]
        
        all_comprehension_df_list.append(comprehension_df)

all_comprehension_df = pd.concat(all_comprehension_df_list, axis = 0)

Ranking Passages for 1 of 4 query/queries
Step 1 : BERT-QE Expansion
	Phase 1
	Phase 2
	Phase 3
Step 2 : Semantic Passage Retrieval
Step 3 : Passage Re-ranking using Fine-Tuned Covid BERT 
Step 4 : Comprehension 
Ranking Passages for 2 of 4 query/queries
Step 1 : BERT-QE Expansion
	Phase 1
	Phase 2
	Phase 3
Step 2 : Semantic Passage Retrieval
Step 3 : Passage Re-ranking using Fine-Tuned Covid BERT 
Step 4 : Comprehension 
Ranking Passages for 3 of 4 query/queries
Step 1 : BERT-QE Expansion
	Phase 1
	Phase 2
	Phase 3
Step 2 : Semantic Passage Retrieval
Step 3 : Passage Re-ranking using Fine-Tuned Covid BERT 
Step 4 : Comprehension 
Ranking Passages for 4 of 4 query/queries
Step 1 : BERT-QE Expansion
	Phase 1
	Phase 2
	Phase 3
Step 2 : Semantic Passage Retrieval
Step 3 : Passage Re-ranking using Fine-Tuned Covid BERT 
Step 4 : Comprehension 
Ranking Passages for 1 of 3 query/queries
Step 1 : BERT-QE Expansion
	Phase 1
	Phase 2
	Phase 3
Step 2 : Semantic Passage Retrieval
Step 3 : Passage

In [27]:
pd.merge(all_comprehension_df, all_metadata_df[["passage_id","passage"]], 
         left_on = "pass_id", right_on = "passage_id", how = "left").to_csv("all_question_comprehension.csv", index = None)